Code to orgenize the bids in the RDOF
Ignacio Nunez, PhD Candidate in Economics, UT-Austin, ijnunez@utexas.edu

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
pd.options.display.max_rows = 30
import matplotlib.pyplot as plt #if using matplotlib
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np
import dbfread
from dbfread import DBF
import linearmodels
from linearmodels import IV2SLS, IVLIML, IVGMM, IVGMMCUE
import statsmodels.api as sm
import csv
import seaborn as sns

# Import datasets

In [22]:
eligible=pd.read_csv('AuctionData/census_blocks.csv')
eligible.rename(columns = {'state':'state_abbreviation', 'census_id':'cbg_id', 'county':'county_name'}, inplace = True)
eligible['partially_eligible']=1
eligible=eligible[['state_abbreviation', 'county_name', 'block_id', 'partially_eligible','cbg_id']]

eligible_cbg=pd.read_csv('AuctionData/all_items.csv')  
bids=pd.read_csv('AuctionData/all_bid_results.csv')  

eligible.columns
eligible_cbg.columns
bids.columns

/Users/ijnunez/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['state_abbreviation', 'county_name', 'block_id', 'partially_eligible',
       'cbg_id'],
      dtype='object')

Index(['auction_id', 'item', 'state', 'county', 'census_id', 'reserve_price',
       'locations'],
      dtype='object')

Index(['auction_id', 'round', 'bidder', 'frn', 'item', 't+l_weight', 'tier',
       'latency', 'price_point_bid', 'bid_clock_pct_flag', 'min_scale_pct',
       'implied_support_bid', 'bid_id', 'my_assigned_status',
       'not_assigned_reason', 'assigned_support', 'assigned_support_type',
       'price_point_assigned', 'assigned_id', 'assigned_id_combined',
       'biddable_next_round', 'package_biddable_next_round', 'state', 'county',
       'census_id', 'locations', 'selection_number'],
      dtype='object')

In [57]:
def getstuff(filename, criterion):
    with open(filename, "rb") as csvfile:
        datareader = csv.reader(csvfile)
        yield next(datareader)  # yield the header row
        count = 0
        for row in datareader:
            if row[3] == criterion:
                yield row
                count += 1
            elif count:
                # done when having read a consecutive series of rows 
                return
chunk = pd.read_csv('Data/Fixed_Broadband_Deployment_Data__December_2020.csv', chunksize=100000)
df = pd.concat(chunk)
def county_tostr(county):
    if county<9999:
        return '0'+str(county)
    else:
        return str(county) 

In [122]:

state='TX'
county='Harris'
state_num=48
county_num=48201

state='PA'
county='Westmoreland'
state_num=42
county_num=42129

In [123]:
bidder='Windstream Holdings, Inc.'
entry_Wind=df[['Census Block FIPS Code','Holding Company Name','Max Advertised Downstream Speed (mbps)',
       'Max Advertised Upstream Speed (mbps)','Technology Code']].loc[df['Holding Company Name']==bidder,:]

entry_Wind=entry_Wind.loc[(entry_Wind['Technology Code']<=90),:]
entry_Wind['entry']=1
entry_Wind['state_num'] = entry_Wind['Census Block FIPS Code'].astype(str).str[0:-13].astype(np.int64)     
entry_Wind['county_num'] = entry_Wind['Census Block FIPS Code'].astype(str).str[0:-10].astype(np.int64)     
entry_Wind['county'] = entry_Wind.apply(lambda x: county_tostr(x['county_num']),axis=1)
entry_Wind['item'] = entry_Wind['Census Block FIPS Code'].astype(str).str[0:-3].astype(np.int64)
entry_Wind['tract'] = entry_Wind['Census Block FIPS Code'].astype(str).str[0:-4].astype(np.int64)

# Generate dataset with last bid by bidder

In [124]:
bids2=bids.loc[lambda df: df['state'] == state, :]
bids2=bids2.loc[lambda df: df['county'] == county, :]

bids2.columns
bids3=bids2[['census_id','bidder','price_point_bid','t+l_weight']]
bids3=bids3.groupby(['census_id','bidder']).min()
bids3=bids3.reset_index()

bids4=bids2[['census_id','bidder','round']]
bids4=bids4.groupby(['census_id','bidder']).max()
bids4=bids4.reset_index()

bids4=bids4.merge(bids3, how='left',left_on=['census_id','bidder'], right_on=['census_id','bidder'])


bids5=bids2.loc[lambda df: df['assigned_support']>0, :]
bids5=bids5[['census_id','bidder','assigned_support','round','price_point_bid','t+l_weight']]

bids6=bids5.merge(bids4, how='right',left_on=['census_id'], right_on=['census_id'])


bids6.describe()
bids6.head()

bids6.to_csv('Westmoreland_bids.csv')

Index(['auction_id', 'round', 'bidder', 'frn', 'item', 't+l_weight', 'tier',
       'latency', 'price_point_bid', 'bid_clock_pct_flag', 'min_scale_pct',
       'implied_support_bid', 'bid_id', 'my_assigned_status',
       'not_assigned_reason', 'assigned_support', 'assigned_support_type',
       'price_point_assigned', 'assigned_id', 'assigned_id_combined',
       'biddable_next_round', 'package_biddable_next_round', 'state', 'county',
       'census_id', 'locations', 'selection_number'],
      dtype='object')

,census_id,assigned_support,round_x,price_point_bid_x,t+l_weight_x,round_y,price_point_bid_y,t+l_weight_y
count,6.260000e+02,563.000000,563.0,563.0,563.000000,626.000000,626.000000,626.000000
mean,4.212980e+11,1679.228135,13.0,60.0,14.174067,11.507987,78.762077,31.325879
std,2.493477e+04,3146.105017,0.0,0.0,9.095279,2.140079,24.900058,18.785080
min,4.212980e+11,12.600000,13.0,60.0,0.000000,7.000000,60.000000,0.000000
25%,4.212980e+11,165.000000,13.0,60.0,0.000000,10.250000,60.000000,20.000000
50%,4.212980e+11,472.320000,13.0,60.0,20.000000,13.000000,62.300000,20.000000
75%,4.212981e+11,1482.400000,13.0,60.0,20.000000,13.000000,89.960000,35.000000
max,4.212981e+11,16756.000000,13.0,60.0,20.000000,14.000000,130.000000,60.000000


,census_id,bidder_x,assigned_support,round_x,price_point_bid_x,t+l_weight_x,bidder_y,round_y,price_point_bid_y,t+l_weight_y
0,421298003002,Space Exploration Technologies Corp.,63.6,13.0,60.0,20.0,Benton Ridge Telephone Company,8,120.00,20
1,421298003002,Space Exploration Technologies Corp.,63.6,13.0,60.0,20.0,"Hughes Network Systems, LLC",13,60.00,35
2,421298003002,Space Exploration Technologies Corp.,63.6,13.0,60.0,20.0,Space Exploration Technologies Corp.,13,60.00,20
3,421298003002,Space Exploration Technologies Corp.,63.6,13.0,60.0,20.0,"Viasat, Inc.",11,80.01,60
4,421298004001,Space Exploration Technologies Corp.,465.2,13.0,60.0,20.0,Benton Ridge Telephone Company,8,120.00,20


# Generate dataset with unserved blocks and spatial location

In [125]:
eligible2=eligible.loc[lambda df: df['state_abbreviation'] == state, :]
eligible2=eligible2.loc[lambda df: df['county_name'] == county, :]

In [126]:
path='GIS/tl_2019_'+str(state_num)+'_tabblock10.dbf'
table=DBF(path)
tiger = pd.DataFrame(data=table)
tiger = tiger[['GEOID10','UR10','ALAND10','INTPTLAT10', 'INTPTLON10']]
tiger['GEOID10']=pd.to_numeric(tiger['GEOID10'], errors='coerce')
tiger = eligible2.merge(tiger, how='inner', left_on='block_id', right_on='GEOID10')
tiger = tiger[['GEOID10','UR10','ALAND10','INTPTLAT10', 'INTPTLON10','cbg_id','state_abbreviation', 'county_name']]
tiger['INTPTLAT10']=pd.to_numeric(tiger['INTPTLAT10'], errors='coerce')
tiger['INTPTLON10']=pd.to_numeric(tiger['INTPTLON10'], errors='coerce')

In [127]:
tiger.head()

,GEOID10,UR10,ALAND10,INTPTLAT10,INTPTLON10,cbg_id,state_abbreviation,county_name
0,421298003002034,U,3601,40.581096,-79.766843,421298003002,PA,Westmoreland
1,421298004001007,U,24676,40.594377,-79.752588,421298004001,PA,Westmoreland
2,421298004001013,U,55808,40.589995,-79.750608,421298004001,PA,Westmoreland
3,421298004001015,U,14856,40.589473,-79.751395,421298004001,PA,Westmoreland
4,421298005002024,U,22224,40.571937,-79.755955,421298005002,PA,Westmoreland


# Generate dataset with incumbent and spatial location

In [128]:
entry_Wind=entry_Wind.loc[lambda df: df['state_num'] == state_num, :]
entry_Wind=entry_Wind.loc[lambda df: df['county_num'] == county_num, :]
entry_Wind.head()

,Census Block FIPS Code,Holding Company Name,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Technology Code,entry,state_num,county_num,county,item,tract
16582852,421298012001005,"Windstream Holdings, Inc.",100.0,8.0,12,1,42,42129,42129,421298012001,42129801200
16582853,421298012001006,"Windstream Holdings, Inc.",100.0,100.0,12,1,42,42129,42129,421298012001,42129801200
16582854,421298012001007,"Windstream Holdings, Inc.",15.0,15.0,12,1,42,42129,42129,421298012001,42129801200
16582855,421298012001008,"Windstream Holdings, Inc.",200.0,200.0,12,1,42,42129,42129,421298012001,42129801200
16582856,421298012001009,"Windstream Holdings, Inc.",15.0,15.0,12,1,42,42129,42129,421298012001,42129801200


In [129]:
entry_Wind_block=entry_Wind.groupby(['Census Block FIPS Code']).max()
entry_Wind_block=entry_Wind_block.reset_index()
entry_Wind_block.head()

,Census Block FIPS Code,Holding Company Name,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Technology Code,entry,state_num,county_num,county,item,tract
0,421298001001016,"Windstream Holdings, Inc.",0.0,0.0,10,1,42,42129,42129,421298001001,42129800100
1,421298004001012,"Windstream Holdings, Inc.",0.0,0.0,11,1,42,42129,42129,421298004001,42129800400
2,421298005002006,"Windstream Holdings, Inc.",0.0,0.0,11,1,42,42129,42129,421298005002,42129800500
3,421298007001016,"Windstream Holdings, Inc.",0.0,0.0,11,1,42,42129,42129,421298007001,42129800700
4,421298012001005,"Windstream Holdings, Inc.",100.0,8.0,12,1,42,42129,42129,421298012001,42129801200


In [130]:
entry_Wind_block.columns

Index(['Census Block FIPS Code', 'Holding Company Name',
       'Max Advertised Downstream Speed (mbps)',
       'Max Advertised Upstream Speed (mbps)', 'Technology Code', 'entry',
       'state_num', 'county_num', 'county', 'item', 'tract'],
      dtype='object')

In [131]:
tiger2 = pd.DataFrame(data=table)
tiger2 = tiger2[['GEOID10','UR10','ALAND10','INTPTLAT10', 'INTPTLON10']]
tiger2['GEOID10']=pd.to_numeric(tiger2['GEOID10'], errors='coerce')
tiger2 = entry_Wind_block.merge(tiger2, how='inner', left_on='Census Block FIPS Code', right_on='GEOID10')
tiger2 = tiger2[['GEOID10','UR10','ALAND10','INTPTLAT10', 'INTPTLON10','Max Advertised Downstream Speed (mbps)',
       'Max Advertised Upstream Speed (mbps)']]
tiger2['INTPTLAT10']=pd.to_numeric(tiger2['INTPTLAT10'], errors='coerce')
tiger2['INTPTLON10']=pd.to_numeric(tiger2['INTPTLON10'], errors='coerce')

# Maps

In [132]:
import plotly.express as px
import plotly.graph_objects as go

In [133]:
bids7=bids6[['census_id','bidder_x','price_point_bid_x']]
bids7=bids7.merge(tiger, how='right',left_on=['census_id'], right_on=['cbg_id'])
bids7['Winner']=bids7['bidder_x']
bids7=bids7.sort_values(by="Winner",ascending=True)
bids7.loc[bids7['Winner'].isna(),'Winner']="Unassigned"
bids7.head()

,census_id,bidder_x,price_point_bid_x,GEOID10,UR10,ALAND10,INTPTLAT10,INTPTLON10,cbg_id,state_abbreviation,county_name,Winner
0,421298003002,Space Exploration Technologies Corp.,60.0,421298003002034,U,3601,40.581096,-79.766843,421298003002,PA,Westmoreland,Space Exploration Technologies Corp.
1316,421298064001,Space Exploration Technologies Corp.,60.0,421298064001006,U,8739,40.107167,-79.599301,421298064001,PA,Westmoreland,Space Exploration Technologies Corp.
1315,421298064001,Space Exploration Technologies Corp.,60.0,421298064001004,U,8742,40.107768,-79.597494,421298064001,PA,Westmoreland,Space Exploration Technologies Corp.
1314,421298064001,Space Exploration Technologies Corp.,60.0,421298064001004,U,8742,40.107768,-79.597494,421298064001,PA,Westmoreland,Space Exploration Technologies Corp.
1313,421298064001,Space Exploration Technologies Corp.,60.0,421298064001004,U,8742,40.107768,-79.597494,421298064001,PA,Westmoreland,Space Exploration Technologies Corp.


In [134]:
bids7['Winner'].unique()

array(['Space Exploration Technologies Corp.',
       'Windstream Services LLC, Debtor-In-Possession', 'Unassigned'],
      dtype=object)

In [135]:
mapbox_access_token = 'pk.eyJ1IjoiaWpudW5leiIsImEiOiJjbDBnMXd5MDMweGJhM2RsY3Q4djAwcmhoIn0.ANDvKIwkfPNoYi-4EcY3YQ'
px.set_mapbox_access_token(mapbox_access_token)
aux=bids7.copy()
aux.loc[aux['Winner']=='Space Exploration Technologies Corp.', 'Winner']="Starlink"
aux.loc[aux['Winner']=='Windstream Services LLC, Debtor-In-Possession', 'Winner']="Windstream"
aux['size']=9
fig = px.scatter_mapbox(aux, lat="INTPTLAT10", lon="INTPTLON10",  color='Winner',color_continuous_scale='Agsunset', mapbox_style='light')
fig.show()

In [136]:
bids7=bids6[['census_id','bidder_y','price_point_bid_y']].loc[lambda df: df['bidder_y']=='Windstream Services LLC, Debtor-In-Possession', :]
bids7=bids7.merge(tiger, how='right',left_on=['census_id'], right_on=['cbg_id'])
bids7['Lowest Bid']=bids7['price_point_bid_y']
bids7=bids7.sort_values(by="Lowest Bid",ascending=True)
#bids7.loc[bids7['Lowest Bid'].isna(),'Lowest Bid']="Did not bid"
bids7=bids7.loc[~bids7['Lowest Bid'].isna(),:]
bids7.head()

,census_id,bidder_y,price_point_bid_y,GEOID10,UR10,ALAND10,INTPTLAT10,INTPTLON10,cbg_id,state_abbreviation,county_name,Lowest Bid
98,4.212980e+11,"Windstream Services LLC, Debtor-In-Possession",60.0,421298021011005,R,63798,40.521427,-79.695740,421298021011,PA,Westmoreland,60.0
101,4.212980e+11,"Windstream Services LLC, Debtor-In-Possession",60.0,421298021013018,R,9012,40.442055,-79.685845,421298021013,PA,Westmoreland,60.0
100,4.212980e+11,"Windstream Services LLC, Debtor-In-Possession",60.0,421298021011040,U,954,40.463324,-79.698472,421298021011,PA,Westmoreland,60.0
99,4.212980e+11,"Windstream Services LLC, Debtor-In-Possession",60.0,421298021011039,U,2682,40.457590,-79.702141,421298021011,PA,Westmoreland,60.0
349,4.212981e+11,"Windstream Services LLC, Debtor-In-Possession",60.0,421298074012001,R,500397,40.375427,-79.426994,421298074012,PA,Westmoreland,60.0


In [137]:
mapbox_access_token = 'pk.eyJ1IjoiaWpudW5leiIsImEiOiJjbDBnMXd5MDMweGJhM2RsY3Q4djAwcmhoIn0.ANDvKIwkfPNoYi-4EcY3YQ'
px.set_mapbox_access_token(mapbox_access_token)
aux=bids7.copy()
aux['size']=9
fig = px.scatter_mapbox(aux, lat="INTPTLAT10", lon="INTPTLON10",  color="Lowest Bid",
                        color_continuous_scale='agsunset', mapbox_style='light')
fig.show()

In [138]:
tiger2.columns

Index(['GEOID10', 'UR10', 'ALAND10', 'INTPTLAT10', 'INTPTLON10',
       'Max Advertised Downstream Speed (mbps)',
       'Max Advertised Upstream Speed (mbps)'],
      dtype='object')

In [139]:
tiger2['Max. Speed']=tiger2['Max Advertised Downstream Speed (mbps)']
tiger2=tiger2.sort_values(by="Max. Speed",ascending=True)
tiger2.loc[tiger2['Max. Speed'].isna(),'Max. Speed']=0
tiger2.describe()
#tiger2=tiger2.loc[lambda df: df['Max. Speed']>0, :]

,GEOID10,ALAND10,INTPTLAT10,INTPTLON10,Max Advertised Downstream Speed (mbps),Max Advertised Upstream Speed (mbps),Max. Speed
count,1.212000e+03,1.212000e+03,1212.000000,1212.000000,1212.000000,1212.000000,1212.000000
mean,4.212980e+14,4.248968e+05,40.440087,-79.596773,163.532178,157.989644,163.532178
std,1.588345e+07,8.892489e+05,0.089627,0.082426,294.344377,296.261952,294.344377
min,4.212980e+14,0.000000e+00,40.127056,-79.845494,0.000000,0.000000,0.000000
25%,4.212980e+14,1.443400e+04,40.390742,-79.656253,15.000000,10.000000,15.000000
50%,4.212980e+14,4.919400e+04,40.419166,-79.614843,50.000000,50.000000,50.000000
75%,4.212980e+14,4.433100e+05,40.467395,-79.567634,100.000000,100.000000,100.000000
max,4.212981e+14,1.092387e+07,40.669760,-79.244626,1000.000000,1000.000000,1000.000000


In [140]:

mapbox_access_token = 'pk.eyJ1IjoiaWpudW5leiIsImEiOiJjbDBnMXd5MDMweGJhM2RsY3Q4djAwcmhoIn0.ANDvKIwkfPNoYi-4EcY3YQ'
px.set_mapbox_access_token(mapbox_access_token)
aux=tiger2.copy()
aux['size']=9
fig = px.scatter_mapbox(aux, lat="INTPTLAT10", lon="INTPTLON10",  color='Max. Speed',
                        color_continuous_scale='bluered',
                        mapbox_style='light')
fig.show()